In [ ]:
# from https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/cgan/cgan.py

import os
os.environ['CUDA_DEVICE_ID'] = 'PCI_BUS_ORDER'
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import argparse
import math
import random

import numpy as np
from tqdm import tqdm
import IPython.display as ipd
from PIL import Image

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets

import torch.nn as nn
import torch.nn.functional as F
import torch

cuda = True if torch.cuda.is_available() else False


students = ['곽승준', '권용순', '김민환', '김봉상', '김용욱', '김형준', '노용문', '박영진', '신용주', '신현준', '안혜영', '오동규', '임항빈', '정근시', '조우성', '조윤정', '홍성규']
CHOICE = random.choice(students)
print(CHOICE)

In [ ]:

MNIST    = 'mnist'        # https://en.wikipedia.org/wiki/MNIST_database
F_MNIST  = 'fashion_mnist' # https://github.com/zalandoresearch/fashion-mnist
CIFAR10 = 'cifar10'     # https://www.cs.toronto.edu/~kriz/cifar.html

DATA_NAME = CIFAR10 


DATA_DIR = f'data/{DATA_NAME}'
GEN_DIR = f'gen_images_{DATA_NAME}'
os.makedirs(GEN_DIR, exist_ok=True)

print(f'torch.version:{torch.__version__}')
print(f'cuda:{cuda}')


!python --version
!pwd
!ls -l

In [ ]:
n_channel = {MNIST:1, F_MNIST:1, CIFAR10:3}[DATA_NAME]
img_size =  {MNIST:28, F_MNIST:28, CIFAR10:32}[DATA_NAME]
VISION_DATASET =  {MNIST:datasets.MNIST, F_MNIST:datasets.FashionMNIST, CIFAR10:datasets.CIFAR10}[DATA_NAME]

class Option:
    def __init__(self):
        self.n_epochs = 50         # number of epochs of training
        self.batch_size = 64       # size of the batches
        self.lr = 0.0002           # adam: learning rate
        self.b1 = 0.5              # adam: decay of first order momentum of gradient
        self.b2 = 0.999            # adam: decay of first order momentum of gradient
        self.n_cpu = 2             # number of cpu threads to use during batch generation
        self.latent_dim = 100      # dimensionality of the latent space
        self.n_classes = 10        # number of classes for dataset
        self.img_size = img_size   # size of each image dimension
        self.channels = n_channel  # number of image channels
        self.sample_interval = 400 # interval between image sampling")
        
opt = Option()
vars(opt)

In [ ]:
img_shape = (opt.channels, opt.img_size, opt.img_size)
models = {}

In [ ]:
# base
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
    
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity
    
models['base'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
# 김형준
# Number of channels in the training images. For color images this is 3
nc = opt.channels

# Size of z latent vector (i.e. size of generator input)
nz = opt.latent_dim

# Size of feature maps in generator
ngf = opt.img_size

# Size of feature maps in discriminator
ndf = opt.img_size

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # https://simpling.tistory.com/entry/Embedding-%EC%9D%B4%EB%9E%80-%EB%AC%B4%EC%97%87%EC%9D%B8%EA%B0%80-%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0
        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        # Linear를 ConvTranspose2d 로 변경
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.ConvTranspose2d(in_feat, out_feat, 4)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat))
            layers.append(nn.ReLU(True))
            return layers
        
        # normalize 를 True 로 BatchNorm2d 를 탈 수 있도록 수정
        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=True),
            *block(128, 256, normalize=True),
            *block(256, 512, normalize=True),
            *block(512, 1024, normalize=True),
            nn.ConvTranspose2d(1024, int(np.prod(img_shape)), 4),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        
        # gen_input 값이 사이즈가 안맞아서 error [64, 110]
        # Expected 4-dimensional input for 4-dimensional weight [110, 128, 4, 4], but got 2-dimensional input of size [64, 110] instead
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
    
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Conv2d(opt.n_classes + int(np.prod(img_shape)), 512, 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, 4),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, 4),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity

models['김형준'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
# 조우성
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_embedding = nn.Sequential(
            nn.Embedding(opt.n_classes, 128),
            nn.Linear(128, 16*4)
        )
        
        self.noise_dense = nn.Sequential(
            nn.Linear(opt.latent_dim, 4*4*256),
            nn.LeakyReLU(0.2, inplace=True)
        )
        
        # ConvTranspose2d(in_c, out_c, k, s, p, bias)
        self.model = nn.Sequential(
            nn.ConvTranspose2d(260, 64*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64*2, momentum=0.1, eps=0.8),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(64*2, 64*1, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64*1, momentum=0.1, eps=0.8),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(64*1, opt.channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        noise = self.noise_dense(noise)
        noise = noise.view(-1, 256, 4, 4)
        
        labels = self.label_embedding(labels)
        labels = labels.view(-1, 4, 4, 4)
        
        concat = torch.cat((noise, labels), dim=1)
        
        img = self.model(concat)

        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_embedding = nn.Sequential(
            nn.Embedding(opt.n_classes, 128),
            nn.Linear(128, np.prod(img_shape))
        )
        
        self.model = nn.Sequential(
            nn.Conv2d(6, 64*1, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64*1, 64*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64*2, momentum=0.1, eps=0.8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64*2, 64*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64*4, momentum=0.1, eps=0.8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Flatten(),
            nn.Dropout(0.6),
            nn.Linear(4096, 128),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        labels = self.label_embedding(labels)
        labels = labels.view(-1, 3, 32, 32)
        
        concat = torch.cat((img, labels), dim=1)
        
        validity = self.model(concat)

        return validity

models['조우성'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.linear_model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 7*7*32, normalize=False)
        )
        self.deconv2d_model = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, 2, 0, bias = False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(16, 8, 3, 1, 1, bias = False),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(8, 3, 4, 2, 0, bias = False)
        )
        self.tanh = nn.Tanh()

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.linear_model(gen_input)
        img = img.view(img.size(0), *(32,7,7))
        img = self.deconv2d_model(img)
        img = self.tanh(img)
        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)        

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Linear((32 * 6 * 6) + 10, 512), 
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(),
            nn.Linear(512, 512), 
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(),
            nn.Linear(512, 1)        
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        validity = self.features(img)
        validity = self.avgpool(validity)
        validity = torch.flatten(validity, 1)
        validity = torch.cat((validity, self.label_embedding(labels)), -1)
        validity = self.classifier(validity)
        return validity

models['오동규'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:

opt_ngf = 64              # size of generator filters
opt_ndf = 64              # size of discriminator filters

class Generator(nn.Module):
    
    def __init__(self):
        super(Generator, self).__init__()

        def conv_block(in_feat, out_feat, kernel_size, stride, padding, normalize=True, activation='leaky_relu'):
            layers = [nn.ConvTranspose2d(in_channels=in_feat, out_channels=out_feat, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat, 0.8))
            activations = {'relu': nn.ReLU(True), 'leaky_relu': nn.LeakyReLU(0.01, inplace=True), 'tanh': nn.Tanh()}
            layers.append(activations[activation])
            return layers
    
        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)   
        self.upsampling = nn.Sequential(
            *conv_block(opt.latent_dim + opt.n_classes, opt_ngf * 4, kernel_size=4, stride=1, padding=0, activation='relu'),
            *conv_block(opt_ngf * 4, opt_ngf * 2, kernel_size=4, stride=2, padding=1, activation='relu'),
            *conv_block(opt_ngf * 2, opt_ngf, kernel_size=4, stride=2, padding=1, activation='relu'),
            *conv_block(opt_ngf, opt.channels, kernel_size=4, stride=2, padding=1, normalize=False, activation='tanh'), # 64 x 3 x 32 x 32
        )

    
    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        for _ in range(2):
            gen_input = torch.unsqueeze(gen_input, -1)
        img = self.upsampling(gen_input)
        return img
    

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity 
    
models['홍성규'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        
        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))
        
        #dcgan Generator 샘플코드에 in_feat, out_feat만 수정
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 512, 3, stride=1, padding=1),
            nn.BatchNorm2d(512, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        # 라벨 임베딩 제거
        # self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)
        
        #dcgan Discriminator샘플에 파라메터 제한에 맞추고 Upsample 추가 in_feat, out_feat 수정
        self.model = nn.Sequential(
            nn.Conv2d(opt.channels, 512, kernel_size=3, padding=0, stride=2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.4),
            nn.BatchNorm2d(512),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=0, stride=2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.4),
            #nn.BatchNorm2d(512),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=0, stride=2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.4),
        )

        
    # 라벨 파라메터 제거
    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        return out
    
    
models['김봉상'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module): #생성자 생성
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)
        #block 정의
        def block(in_feat, out_feat, normalize=True): #선형함수 
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                #배치 정규화실행 (차원 동일)
                #activation function 은 leakyRelu 사용
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        #생성자 모델은 연속적인 여러개의 블럭을 가짐
        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))), # 결과적으로 1024 
            nn.Tanh() # tanh 사용하여 -1 ~ 1 사이에 값 배출
        )

    def forward(self, noise, labels):
        # 노이즈함수, label값 뱉어냄
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape) #view 함수 사용하여 img 형태를 가질 수 있게함
        return img
    
class Discriminator(nn.Module): #판별자 생성
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512), #generator와는 다르게 linear함수 사용
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1), 
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity
    
models['김용욱'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            # 입력값은 Z이며 Transposed Convolution을 거칩니다.
            nn.ConvTranspose2d(opt.latent_dim + opt.n_classes, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            # (512) x 4 x 4
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            # (256) x 8 x 8
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            # (128) x 16 x 16
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            # 64 x 32 x 32
            nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=False),
            nn.Tanh()
        )
            
    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
    
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
     
            # (64) x 64 x 64)
            nn.Conv2d(opt.n_classes + int(np.prod(img_shape)), 64, 4,2,1,bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # 128 x 32 x 32
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            # (256 * 2) x 16 x 16
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            # (512 * 4) x 8 x 8
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()

        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity
    
models['노용문'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)
        self.latent_embedding = nn.Sequential(
            nn.Linear(opt.latent_dim, 100),#32*32 // 2),
        )
        self.condition_embedding = nn.Sequential(
            nn.Linear(opt.n_classes, 10),#32*32 // 2),
        )
        
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        
        self.model = nn.Sequential(
            
            
            # sample model. It has nn.ConvTranspose2d(1, 3, 4, 1, 0, bias = False)
            # First parameter = Channels of input (=1)
            # Second parameter = Channels of output (=3)
            # Third parameter = Kernel size (=4)
            # Fourth parameter = stride (=1)
            # fifth parameter = padding (=0)
            
            # (in - 1)*stride - 2*padding + (kernel-1) +1
            
            nn.ConvTranspose2d( 110, 32*32, 4, 1, 0),
            nn.BatchNorm2d(32*32),
            nn.LeakyReLU(0.2,inplace=True),
            #torch.Size([64, 1024, 4, 4])
            nn.ConvTranspose2d( 32*32, 32*32 // 2, 3, 2, 1),
            nn.BatchNorm2d(32*32 // 2),
            nn.LeakyReLU(0.2,inplace=True),
            #torch.Size([64, 512, 7, 7])
            
            nn.ConvTranspose2d( 32*32 // 2, 32*32 // 4, 2, 2, 1),
            nn.BatchNorm2d(32*32 // 4),
            nn.LeakyReLU(0.2,inplace=True),
            #torch.Size([64, 256, 12, 12])
            
            nn.ConvTranspose2d( 32*32 // 4, 32*32 // 8, 3, 1, 1),
            nn.BatchNorm2d(32*32 // 8),
            nn.LeakyReLU(0.2,inplace=True),
            #torch.Size([64, 128, 12, 12])
            
            nn.ConvTranspose2d( 32*32 // 8, 32*32 // 16, 3, 2, 1),
            nn.BatchNorm2d(32*32 // 16),
            nn.LeakyReLU(0.2,inplace=True),
            #torch.Size([64, 64, 23, 23])
            
            nn.ConvTranspose2d( 32*32 // 16, 32*32 // 32, 4, 1, 0),
            nn.BatchNorm2d(32*32 // 32),
            nn.LeakyReLU(0.2,inplace=True),
            #torch.Size([64, 32, 26, 26])
            
            nn.ConvTranspose2d( 32*32 // 32, 32*32 // 64, 4, 1, 0),
            nn.BatchNorm2d(32*32 // 64),
            nn.LeakyReLU(0.2,inplace=True),
            #torch.Size([64, 16, 29, 29])
            
            #nn.ConvTranspose2d( 32*32 // 64, 1, 4, 1, 0),
            nn.ConvTranspose2d( 32*32 // 64, opt.channels, 4, 1, 0),
            #torch.Size([64, 3, 32, 32])
            
            #nn.ConvTranspose2d( 32*32 // 32, opt.channels, 3, 1, 0),
            
            nn.Tanh()
            
            
        )
        
        self.cnn_model = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 4, stride=1, padding='same'),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d( 128, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2,inplace=True),
            
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 5, stride=1, padding='same'),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d( 128, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2,inplace=True),
            
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 5, stride=1, padding='same'),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 5, stride=1, padding='same'),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(in_channels = 128, out_channels = 3, kernel_size = 5, stride=1, padding='same'),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        #gen_input = torch.cat((self.label_emb(labels), noise), -1)
        #img = self.model(gen_input)
        #img = img.view(img.size(0), *img_shape)
        ################################################################
        vec_latent = self.latent_embedding(noise)
        vec_class = self.condition_embedding(self.label_emb(labels))
        combined = torch.cat([vec_latent,vec_class], dim=1).reshape(-1, 110, 1, 1)
        
        #print(combined.shape)
        img = self.model(combined)
        ##################################################################
        #gen_input = torch.cat((self.label_emb(labels), noise), -1)
        
        #img = self.model(gen_input)
        #img = img.reshape(img.shape[0],128,8,8)
        #img = self.cnn_model(img)
        
        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)
        
        self.cnn_model = nn.Sequential(
            #nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 5, stride=1, padding=0), 
            nn.Conv2d(in_channels = opt.channels, out_channels = 6, kernel_size = 5, stride=1, padding=0), 
            #output size = (width-kernel+2*padding)/stride +1
            #cifar10 : (32-5+2*0)/1+1 = 28
            #MNIST : (28-5+2*0)/1+1 = 24
            nn.MaxPool2d(2, 2),
            #28/2 = 14
            #12
            nn.Conv2d(6, 16, 5),
            #(14-5+2*0)/1+1 = 10
            #(12-5+2*0)/1+1 = 8
            nn.MaxPool2d(2, 2),
            #pooling 5
            #4     
            
        )


        self.model = nn.Sequential(
            #######################################################################
            
            #nn.Linear(16 * 4 * 4+opt.n_classes, 512),
            
            nn.Linear(16 * 5 * 5+opt.n_classes, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1)
            
        )

    def forward(self, img, labels):
        
        x = self.cnn_model(img)
        x = torch.cat((x.view(x.size(0), -1), self.label_embedding(labels)), -1)
        #print(x.shape)
        x = self.model(x)        

        validity = x
        return validity
    
    

models['박영진'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)
        
        self.model = nn.Sequential(
            # 입력데이터 Z가 가장 처음 통과하는 전치 합성곱 계층입니다.
            nn.ConvTranspose2d( opt.latent_dim, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),
            # 위의 계층을 통과한 데이터의 크기. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),
            # 위의 계층을 통과한 데이터의 크기. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( 64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),
            # 위의 계층을 통과한 데이터의 크기. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( 64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # 위의 계층을 통과한 데이터의 크기. (ngf) x 32 x 32
            nn.ConvTranspose2d( 64, 3 , 4, 2, 1, bias=False),
            nn.Tanh()
            # 위의 계층을 통과한 데이터의 크기. (nc) x 64 x 64
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        #self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            # 입력 데이터의 크기는 (nc) x 64 x 64 입니다
            nn.Conv2d(opt.channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # 위의 계층을 통과한 데이터의 크기. (ndf) x 32 x 32
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # 위의 계층을 통과한 데이터의 크기. (ndf*2) x 16 x 16
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # 위의 계층을 통과한 데이터의 크기. (ndf*4) x 8 x 8
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # 위의 계층을 통과한 데이터의 크기. (ndf*8) x 4 x 4
            nn.Conv2d(64 * 8, 1 , 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity    

models['신용주'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.ConvTranspose2d(in_feat, out_feat, 5)]
            #layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                # BatchNorm2d(N, C, H, W)
                layers.append(nn.BatchNorm2d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.ConvTranspose2d(1024, int(np.prod(img_shape)), 5),
            #nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        gen_input = gen_input.unsqueeze(2).unsqueeze(3)
        #print(gen_input.shape) # torch.Size([64, 110])
        #img = self.model(gen_input)
        #img = img.view(img.size(0), *img_shape)
        #return img
        return self.model(gen_input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            # nn.Conv2d(in_channels, out_channels, kernel_size, kernel_size, stride, padding) 
            # 필수 파라미터 : in_channels, out_channels, kernel_size(kernel mean is same filter)
            #nn.Conv2d(opt.channels, opt.img_size, 5),
            #nn.LeakyReLU(0.2),
            #nn.Conv2d(opt.img_size, opt.n_classes + int(np.prod(img_shape)), 5),
            #nn.LeakyReLU(0.2),
            nn.Conv2d(opt.n_classes + int(np.prod(img_shape)), 512, 5),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 256, 5),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 128, 5),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 1, 5),
            #nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(512, 512),
            #nn.Dropout(0.4),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(512, 512),
            #nn.Dropout(0.4),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        print(img.shape)
        print(labels.shape)
        d_in = d_in.unsqueeze(2).unsqueeze(3)
        validity = self.model(d_in)
        return validity

models['조윤정'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)
        d = 128
        self.model = nn.Sequential(
            #nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(512, 512),
            #nn.Dropout(0.4),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(512, 512),
            #nn.Dropout(0.4),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(512, 1),            
            nn.Conv2d(1, d, 4, 2, 1),
            nn.Conv2d(d, d*2, 4, 2, 1),
            nn.BatchNorm2d(d*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d*2, d*4, 4, 2, 1),
            nn.BatchNorm2d(d*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d*4, d*8, 4, 2, 1),
            nn.BatchNorm2d(d*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d*8, 1, 4, 1, 0),            
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity   
    
models['정근시'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Linear(110, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)
        
        self.model = nn.Sequential(
            nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity
    
models['안혜영'] = {'G':Generator(), 'D': Discriminator()}

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Conv2d(in_feat, out_feat)]
            if normalize:
                layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Conv2d(1024, int(np.prod(img_shape))),
            nn.MaxPool2d(kernel_size=2,stride=2))
            
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),                                              
            nn.ReLU(),
            # [100,100] -> [100,10]
            nn.Linear(100,10)
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
            
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Conv2d(opt.n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
            nn.LeakyReLU(0.2, inplace=True),
            # [100,100] -> [100,10]
            nn.Linear(100,10)
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity        
            
try:
    models['김민환'] = {'G':Generator(), 'D': Discriminator()}
except:
    pass

In [ ]:
def  num_params(m):
    return sum(p.numel() for p in m.parameters())

    
model = models[CHOICE]
generator = model['G']
discriminator = model['D']


print(num_params(generator), num_params(models['base']['G']))
print(num_params(discriminator), num_params(models['base']['D']))

In [ ]:
# Loss functions
adversarial_loss = torch.nn.MSELoss()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    

In [ ]:
# Configure data loader

os.makedirs(DATA_DIR, exist_ok=True)

ds = VISION_DATASET(DATA_DIR, train=True,download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), 
             transforms.ToTensor(), 
             transforms.Normalize([0.5], [0.5])
            ]
        ),
    )

dataloader = torch.utils.data.DataLoader(
    ds,
    batch_size=opt.batch_size,
    shuffle=True
)

In [ ]:
# display sample
#for _ in range(10):
#    img, label =  random.choice(ds)
#    img = ((img + 1)/2*255).type(torch.uint8).permute((1,2,0)).squeeze()
#    print(f'[{label}] {ds.classes[label]}')
#    display(Image.fromarray(img.numpy()).resize((128, 128)))

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [ ]:
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

def sample_image(n_width, epoch, n_label = 10):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = torch.Tensor(np.random.normal(0, 1, (n_width* n_label, opt.latent_dim))).type(FloatTensor)
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([label for label in range(n_label) for _ in range(n_width)])
    labels = torch.Tensor(labels).type(LongTensor)
    gen_imgs = generator(z, labels)
    img_path = f"{GEN_DIR}/{epoch:06d}.png"
    save_image(gen_imgs.data, img_path, nrow=n_width, normalize=True)
    return img_path


In [ ]:
# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    print(f'{epoch+1}/{opt.n_epochs}')
    if epoch:
        print(f'D loss: {d_loss.item():.04f},  G loss: {g_loss.item()}')
        display(Image.open(img_path))
        
        
    for i, (imgs, labels) in enumerate(tqdm(dataloader)):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = FloatTensor(batch_size, 1).fill_(1.0)
        fake  = FloatTensor(batch_size, 1).fill_(0.0)

        # Configure input
        real_imgs = imgs.type(FloatTensor)
        labels = labels.type(LongTensor)

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim)))
        gen_labels = LongTensor(np.random.randint(0, opt.n_classes, batch_size))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

    img_path = sample_image(n_width=30, epoch=epoch, n_label=opt.n_classes)
    ipd.clear_output(wait=True)